In [115]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?![\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,receive,reach,consent,grieve,lesson,denial,dinah'll,danger,adoption,cut,...,convince,ours,edgar,cur,continually,mystery,moderate,duty,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [7]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9918495297805643

Test set score: 0.8956766917293233


In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 3062) (3190,)
Training set score: 0.9579937304075236

Test set score: 0.9158834586466166


In [9]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8846394984326019

Test set score: 0.8731203007518797


In [10]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [11]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [12]:
# Group into sentences.
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [13]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [92]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((X_train[y_train[y_train=='Carroll'].index],
                              emma_bow.drop(['text_sentence','text_source'], 1)),
                             axis=0)

y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.6976137211036539


col_0,Austen,Carroll
row_0,,
Austen,1564,105
Carroll,706,307


In [15]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [21]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)
train = knn.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', knn.score(X_train, y_train))
print('\nTest set score:', knn.score(X_test, y_test))
print('\nEmma test set score:', knn.score(X_Emma_test, y_Emma_test))

(3190, 3062) (3190,)
Training set score: 0.8727272727272727

Test set score: 0.7584586466165414

Emma test set score: 0.56972408650261


In [22]:
from sklearn.svm import LinearSVC

svc = LinearSVC()
train = svc.fit(X_train, y_train)
print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))
print('\nEmma test set score:', svc.score(X_Emma_test, y_Emma_test))

Training set score: 0.9843260188087775

Test set score: 0.8980263157894737

Emma test set score: 0.6733780760626398


## Make new features

In [110]:
counts = []
for x in sentences[0]:
    counts.append(len(x))

In [112]:
sentences['word_count'] = counts

In [138]:
punc_counts = []

for x in sentences[0]:
    puncts = 0
    for token in x:
        if token.is_punct:
            puncts =+ 1
    punc_counts.append(puncts)

In [139]:
sentences['punc_counts'] = punc_counts

In [142]:
sentences.punc_counts.value_counts()

1    5282
0      36
Name: punc_counts, dtype: int64

In [187]:
sentences['entities'] = 0
for x in sentences[0]:
    ents = []
    for ent in x:
        ents.append(ent)
    sentences['entities'] = ents
       

ValueError: Length of values does not match length of index

In [184]:
sentences.head()

,0,1,word_count,punc_counts,previous_sent_count,entities
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,67,1,66,Finis
1,"(So, she, was, considering, in, her, own, mind...",Carroll,63,1,62,Finis
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,33,1,32,Finis
3,"(Oh, dear, !)",Carroll,3,1,2,Finis
4,"(I, shall, be, late, !, ')",Carroll,6,1,5,Finis


# checking model against new text

In [102]:
burgess = gutenberg.raw('burgess-busterbrown.txt')
#burgess = re.sub(r'VOLUME \w+', '', burgess)
#burgess = re.sub(r'CHAPTER \w+', '', burgess)
#burgess = text_cleaner(burgess)



In [103]:
burgess

'[The Adventures of Buster Bear by Thornton W. Burgess 1920]\r\n\r\nI\r\n\r\nBUSTER BEAR GOES FISHING\r\n\r\n\r\nBuster Bear yawned as he lay on his comfortable bed of leaves and\r\nwatched the first early morning sunbeams creeping through the Green\r\nForest to chase out the Black Shadows. Once more he yawned, and slowly\r\ngot to his feet and shook himself. Then he walked over to a big\r\npine-tree, stood up on his hind legs, reached as high up on the trunk of\r\nthe tree as he could, and scratched the bark with his great claws. After\r\nthat he yawned until it seemed as if his jaws would crack, and then sat\r\ndown to think what he wanted for breakfast.\r\n\r\nWhile he sat there, trying to make up his mind what would taste best, he\r\nwas listening to the sounds that told of the waking of all the little\r\npeople who live in the Green Forest. He heard Sammy Jay way off in the\r\ndistance screaming, "Thief! Thief!" and grinned. "I wonder," thought\r\nBuster, "if some one has stolen S

In [95]:
# Parse our cleaned data.
burgess_doc = nlp(burgess)

In [98]:
# Group into sentences.
burgess_sents = [[burgess, "Burgess"] for sent in burgess_doc.sents]

# make sure everything is the same size as alice
burgess_sents = burgess_sents[0:len(alice_sents)]

In [99]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
burgess_sentences = pd.DataFrame(burgess_sents)
burgess_bow = bow_features(burgess_sentences, common_words)

print('done')

AttributeError: 'str' object has no attribute 'is_punct'

In [93]:
X_burgess_test = np.concatenate((X_train[y_train[y_train=='Carroll'].index],
                              sburgess_bow.drop(['text_sentence','text_source'], 1)),
                             axis=0)
y_burgess_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Burgess'] * burgess_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_burgess_test, y_burgess_test))
lr_burgess_predicted = lr.predict(X_burgess_test)
pd.crosstab(y_burgess_test, lr_burgess_predicted)


Test set score: 0.6823266219239373


col_0,Austen,Carroll
row_0,,
Austen,1523,146
Carroll,706,307
